In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean
from scipy import stats

In [ ]:
results_folder = os.getcwd()
csv_details = "EXPERIMENTS.csv"
csv_explicit = "explicit.csv"
csv_implicit = "implicit.csv"
csv_init = "init.csv"

folder_plots = "plots/trust_extra"

In [ ]:
exp_details = pd.read_csv(results_folder+"/"+csv_details)
survey_explicit = pd.read_csv(results_folder+"/"+csv_explicit)
survey_implicit = pd.read_csv(results_folder+"/"+csv_implicit)
survey_init = pd.read_csv(results_folder+"/"+csv_init)

# Isolate columns that we want
for i in range(17):
    survey_init.drop(survey_init.columns[[0]], axis=1, inplace=True)

for i in range(15):
    survey_implicit.drop(survey_implicit.columns[[0]], axis=1, inplace=True)
    survey_explicit.drop(survey_explicit.columns[[0]], axis=1, inplace=True)
    
for i in range(24):
    survey_implicit.drop(survey_implicit.columns[[-1]], axis=1, inplace=True)
    survey_explicit.drop(survey_explicit.columns[[-1]], axis=1, inplace=True)

In [ ]:
def get_trust_scores(df_survey, col):
    trust_scores = []
    for index in range(len(df_survey)):
        row = df_survey.iloc[index].tolist()
        # From percentage (xx%) to float (0.xx)
        trust_index = float(row[col].replace('%', 'e-2'))
        # REVERSE: è incompetente
        #if col == 7:
        #    trust_index = 1-trust_index
        trust_scores.append(np.round(trust_index, 2))     
    return trust_scores

def update_ax_mean(df, ax, indexes_1, indexes_2, color_1, color_2):
    indexes = list(set(indexes_1).intersection(set(indexes_2)))
    if len(indexes) > 1:
        init_mean = np.mean(df["INIT_TRUST"][indexes].tolist())*100
        first_mean = np.mean(df["FIRST_TRUST"][indexes].tolist())*100
        second_mean = np.mean(df["SECOND_TRUST"][indexes].tolist())*100
        ax.plot(["INIT", "FIRST"], [init_mean, first_mean], color_1+'o--', markersize=7)
        ax.plot(["FIRST", "SECOND"], [first_mean, second_mean], color_2+'o--', markersize=7)
    return len(indexes)
    

color_true_pred = "g"
color_younger_pred = "b"
color_older_pred = "r"

In [ ]:
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

legend_elements = [Line2D([0], [0], color=color_true_pred, linestyle="--", label="Right age group"),
                   Line2D([0], [0], color=color_younger_pred, linestyle="--", label="Younger age group"),
                   Line2D([0], [0], color=color_older_pred, linestyle="--", label="Older age group or wrong gender")
                  ]

In [ ]:
questions = [
"Comunica in modo chiaro",
"Comunica apertamente",
"Esegue un compito meglio di un utente umano inesperto",
"Mantiene al sicuro le informazioni classificate",
"Prende decisioni sensate",
"Dice la verità",
"È responsabile",
"È incompetente",
"È amichevole",
"È piacevole",
"È autonomo",
"È consapevole"]


In [ ]:
for i in range(12): # for each question
    question = i
    trust_init_scores = get_trust_scores(survey_init, question)
    trust_implicit_scores = get_trust_scores(survey_implicit, question)
    trust_explicit_scores = get_trust_scores(survey_explicit, question)

    # Create list of first and second surveys (explicit, implicit)
    first_surveys = []
    second_surveys = []
    for index in np.unique(exp_details['ID']):
        order_behaviors = exp_details[exp_details['ID'] == index]['BEHAVIOR'].tolist()
        first_surveys.append(order_behaviors[0])
        second_surveys.append(order_behaviors[1])

    # Create list with trust scores based on survey done
    first_trust = []
    second_trust = []
    for i in range(len(first_surveys)):
        if first_surveys[i] == 'explicit' and second_surveys[i] == 'implicit':
            first_trust.append(trust_explicit_scores[i])
            second_trust.append(trust_implicit_scores[i])
        elif first_surveys[i] == 'implicit' and second_surveys[i] == 'explicit':
            first_trust.append(trust_implicit_scores[i])
            second_trust.append(trust_explicit_scores[i])
        else:
            print("ERROR: in explicit or implicit trust sequence.")
    # DF with all data
    df = pd.DataFrame(np.unique(exp_details['ID']), columns = ['ID'] )
    df['INIT_TRUST'] = trust_init_scores
    df['FIRST_SURVEY'] = first_surveys
    df['FIRST_TRUST'] = first_trust
    df['SECOND_SURVEY'] = second_surveys
    df['SECOND_TRUST'] = second_trust
    # Dataframe with sequence explicit and then implicit
    df_ei = df[df['FIRST_SURVEY'] == "explicit"]
    # Dataframe with sequence implicit and then explicit
    df_ie = df[df['FIRST_SURVEY'] == "implicit"]
    
# ------------------------E+I--------------------------------------
    # WRONG PREDICTION SAID IN THE first PART WITH explicit BEHAVIOR
    id_true_pred = [0, 4, 6, 8, 10, 14, 16, 18, 20, 24, 26, 28, 36, 40, 42, 44, 46, 50, 54]
    id_younger_pred = [34, 48, 52, 56, 60]
    id_older_pred = [2, 12, 22, 30, 32, 38, 58]

    # WRONG PREDICTION IN THE second PART WITH implicit BEHAVIOR
    id_true_pred_2 = [4, 6, 8, 10, 12, 16, 20, 24, 28, 30, 38, 40, 42, 44, 46, 50, 54, 58, 60]
    id_younger_pred_2 = [0, 18, 26, 34, 36, 48, 52, 56]
    id_older_pred_2 = [2, 14, 22, 32]

    fig, ax = plt.subplots(1,1,figsize=(15, 10)) 

    numerosity = {}

    numerosity['true-true'] = update_ax_mean(df_ei, ax, id_true_pred, id_true_pred_2, color_true_pred, color_true_pred)
    numerosity['true-younger'] = update_ax_mean(df_ei, ax, id_true_pred, id_younger_pred_2, color_true_pred, color_younger_pred)
    numerosity['true-older'] = update_ax_mean(df_ei, ax, id_true_pred, id_older_pred_2, color_true_pred, color_older_pred)

    numerosity['younger-true'] = update_ax_mean(df_ei, ax, id_younger_pred, id_true_pred_2, color_younger_pred, color_true_pred)
    numerosity['younger-younger'] = update_ax_mean(df_ei, ax, id_younger_pred, id_younger_pred_2, color_younger_pred, color_younger_pred)
    numerosity['younger-older'] = update_ax_mean(df_ei, ax, id_younger_pred, id_older_pred_2, color_younger_pred, color_older_pred)

    numerosity['older-true'] = update_ax_mean(df_ei, ax, id_older_pred, id_true_pred_2, color_older_pred, color_true_pred)
    numerosity['older-younger'] = update_ax_mean(df_ei, ax, id_older_pred, id_younger_pred_2, color_older_pred, color_younger_pred)
    numerosity['older-older'] = update_ax_mean(df_ei, ax, id_older_pred, id_older_pred_2, color_older_pred, color_older_pred)
    
    plt.yticks(np.arange(0, 105, 5))
    plt.grid(axis='y')
    ax.legend(handles=legend_elements, loc='lower left')
    plt.title("Il robot: "+questions[question])
    plt.savefig(folder_plots+"/Q"+str(question)+"_ei.jpg")

# ------------------------I+E------------------------------
    id_true_pred_1_all = id_true_pred
    id_younger_pred_1_all = id_younger_pred
    id_older_pred_1_all = id_older_pred

    id_true_pred_2_all = id_true_pred_2
    id_younger_pred_2_all = id_younger_pred_2
    id_older_pred_2_all = id_older_pred_2

    # WRONG PREDICTION IN THE first PART WITH implicit BEHAVIOR
    id_true_pred = [3, 5, 9, 13, 15, 17, 19, 23, 25, 31, 33, 37, 41, 43, 47, 51]
    id_younger_pred = [29, 39, 45, 53, 55, 59, 61]
    id_older_pred = [1, 7, 11, 21, 27, 35, 49, 57]

    # WRONG PREDICTION SAID IN THE second PART WITH explicit BEHAVIOR
    id_true_pred_2 = [1, 5, 9, 11, 13, 17, 19, 23, 25, 31, 33, 35, 37, 41, 43, 47, 49, 51, 57]
    id_younger_pred_2 = [29, 39, 45, 53, 55, 59, 61]
    id_older_pred_2 = [3, 7, 15, 21, 27]

    fig, ax = plt.subplots(1,1,figsize=(15, 10)) 

    numerosity = {}

    numerosity['true-true'] = update_ax_mean(df_ie, ax, id_true_pred, id_true_pred_2, color_true_pred, color_true_pred)
    numerosity['true-younger'] = update_ax_mean(df_ie, ax, id_true_pred, id_younger_pred_2, color_true_pred, color_younger_pred)
    numerosity['true-older'] = update_ax_mean(df_ie, ax, id_true_pred, id_older_pred_2, color_true_pred, color_older_pred)

    numerosity['younger-true'] = update_ax_mean(df_ie, ax, id_younger_pred, id_true_pred_2, color_younger_pred, color_true_pred)
    numerosity['younger-younger'] = update_ax_mean(df_ie, ax, id_younger_pred, id_younger_pred_2, color_younger_pred, color_younger_pred)
    numerosity['younger-older'] = update_ax_mean(df_ie, ax, id_younger_pred, id_older_pred_2, color_younger_pred, color_older_pred)

    numerosity['older-true'] = update_ax_mean(df_ie, ax, id_older_pred, id_true_pred_2, color_older_pred, color_true_pred)
    numerosity['older-younger'] = update_ax_mean(df_ie, ax, id_older_pred, id_younger_pred_2, color_older_pred, color_younger_pred)
    numerosity['older-older'] = update_ax_mean(df_ie, ax, id_older_pred, id_older_pred_2, color_older_pred, color_older_pred)
    
    plt.yticks(np.arange(0, 105, 5))
    plt.grid(axis='y')
    ax.legend(handles=legend_elements, loc='lower left')
    plt.title("Il robot: "+questions[question])
    plt.savefig(folder_plots+"/Q"+str(question)+"_ie.jpg")

# ------------------------ALL------------------------------
    id_true_pred_1_all = np.unique(id_true_pred_1_all+id_true_pred)
    id_younger_pred_1_all = np.unique(id_younger_pred_1_all+id_younger_pred)
    id_older_pred_1_all = np.unique(id_older_pred_1_all+id_older_pred)

    id_true_pred_2_all = np.unique(id_true_pred_2_all+id_true_pred_2)
    id_younger_pred_2_all = np.unique(id_younger_pred_2_all+id_younger_pred_2)
    id_older_pred_2_all = np.unique(id_older_pred_2_all+id_older_pred_2)

    fig, ax = plt.subplots(1,1,figsize=(15, 10)) 

    numerosity = {}

    numerosity['true-true'] = update_ax_mean(df, ax, id_true_pred_1_all, id_true_pred_2_all, color_true_pred, color_true_pred)
    numerosity['true-younger'] = update_ax_mean(df, ax, id_true_pred_1_all, id_younger_pred_2_all, color_true_pred, color_younger_pred)
    numerosity['true-older'] = update_ax_mean(df, ax, id_true_pred_1_all, id_older_pred_2_all, color_true_pred, color_older_pred)

    numerosity['younger-true'] = update_ax_mean(df, ax, id_younger_pred_1_all, id_true_pred_2_all, color_younger_pred, color_true_pred)
    numerosity['younger-younger'] = update_ax_mean(df, ax, id_younger_pred_1_all, id_younger_pred_2_all, color_younger_pred, color_younger_pred)
    numerosity['younger-older'] = update_ax_mean(df, ax, id_younger_pred_1_all, id_older_pred_2_all, color_younger_pred, color_older_pred)

    numerosity['older-true'] = update_ax_mean(df, ax, id_older_pred_1_all, id_true_pred_2_all, color_older_pred, color_true_pred)
    numerosity['older-younger'] = update_ax_mean(df, ax, id_older_pred_1_all, id_younger_pred_2_all, color_older_pred, color_younger_pred)
    numerosity['older-older'] = update_ax_mean(df, ax, id_older_pred_1_all, id_older_pred_2_all, color_older_pred, color_older_pred)
    
    plt.yticks(np.arange(0, 105, 5))
    plt.grid(axis='y')
    ax.legend(handles=legend_elements, loc='lower left')
    plt.title("Il robot: "+questions[question])
    plt.savefig(folder_plots+"/Q"+str(question)+"_both.jpg")